In [1]:
import streamlit as st
import faiss

from io import BytesIO
import numpy as np

from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_text_splitters import CharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings

from langchain_community.docstore.in_memory import InMemoryDocstore

from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.llms import HuggingFacePipeline

from langchain_huggingface import HuggingFaceEndpoint
import sentence_transformers

C:\Users\Lakshya\AppData\Roaming\Python\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import os
from dotenv import load_dotenv
load_dotenv()
HUGGINGFACE_API_TOKEN = os.getenv("HUGGINGFACE_API_TOKEN")

In [3]:
documents = '''# Build a RAG agent with LangChain

## Overview

One of the most powerful applications enabled by LLMs is sophisticated question-answering (Q\&A) chatbots. These are applications that can answer questions about specific source information. These applications use a technique known as Retrieval Augmented Generation, or [RAG](/oss/python/langchain/retrieval/).

This tutorial will show how to build a simple Q\&A application over an unstructured text data source. We will demonstrate:

1. A RAG [agent](#rag-agents) that executes searches with a simple tool. This is a good general-purpose implementation.
2. A two-step RAG [chain](#rag-chains) that uses just a single LLM call per query. This is a fast and effective method for simple queries.

### Concepts

We will cover the following concepts:

* **Indexing**: a pipeline for ingesting data from a source and indexing it. *This usually happens in a separate process.*

* **Retrieval and generation**: the actual RAG process, which takes the user query at run time and retrieves the relevant data from the index, then passes that to the model.

Once we've indexed our data, we will use an [agent](/oss/python/langchain/agents) as our orchestration framework to implement the retrieval and generation steps.

<Note>
  The indexing portion of this tutorial will largely follow the [semantic search tutorial](/oss/python/langchain/knowledge-base).

  If your data is already available for search (i.e., you have a function to execute a search), or you're comfortable with the content from that tutorial, feel free to skip to the section on [retrieval and generation](#2-retrieval-and-generation)
</Note>

### Preview

In this guide we'll build an app that answers questions about the website's content. The specific website we will use is the [LLM Powered Autonomous Agents](https://lilianweng.github.io/posts/2023-06-23-agent/) blog post by Lilian Weng, which allows us to ask questions about the contents of the post.

We can create a simple indexing pipeline and RAG chain to do this in \~40 lines of code. See below for the full code snippet:

<Accordion title="Expand for full code snippet">
  ```python  theme={null}
  import bs4
  from langchain.agents import AgentState, create_agent
  from langchain_community.document_loaders import WebBaseLoader
  from langchain.messages import MessageLikeRepresentation
  from langchain_text_splitters import RecursiveCharacterTextSplitter

  # Load and chunk contents of the blog
  loader = WebBaseLoader(
      web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
      bs_kwargs=dict(
          parse_only=bs4.SoupStrainer(
              class_=("post-content", "post-title", "post-header")
          )
      ),
  )
  docs = loader.load()

  text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
  all_splits = text_splitter.split_documents(docs)

  # Index chunks
  _ = vector_store.add_documents(documents=all_splits)

  # Construct a tool for retrieving context
  @tool(response_format="content_and_artifact")
  def retrieve_context(query: str):
      """Retrieve information to help answer a query."""
      retrieved_docs = vector_store.similarity_search(query, k=2)
      serialized = "\n\n".join(
          (f"Source: {doc.metadata}\nContent: {doc.page_content}")
          for doc in retrieved_docs
      )
      return serialized, retrieved_docs

  tools = [retrieve_context]
  # If desired, specify custom instructions
  prompt = (
      "You have access to a tool that retrieves context from a blog post. "
      "Use the tool to help answer user queries."
  )
  agent = create_agent(model, tools, system_prompt=prompt)
  ```

  ```python  theme={null}
  query = "What is task decomposition?"
  for step in agent.stream(
      {"messages": [{"role": "user", "content": query}]},
      stream_mode="values",
  ):
      step["messages"][-1].pretty_print()
  ```

  ```
  ================================ Human Message =================================

  What is task decomposition?
  ================================== Ai Message ==================================
  Tool Calls:
    retrieve_context (call_xTkJr8njRY0geNz43ZvGkX0R)
   Call ID: call_xTkJr8njRY0geNz43ZvGkX0R
    Args:
      query: task decomposition
  ================================= Tool Message =================================
  Name: retrieve_context

  Source: {'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}
  Content: Task decomposition can be done by...

  Source: {'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}
  Content: Component One: Planning...
  ================================== Ai Message ==================================

  Task decomposition refers to...
  ```

  Check out the [LangSmith trace](https://smith.langchain.com/public/a117a1f8-c96c-4c16-a285-00b85646118e/r).
</Accordion>

## Setup

### Installation

This tutorial requires these langchain dependencies:

<CodeGroup>
  ```bash pip theme={null}
  pip install langchain langchain-text-splitters langchain-community bs4
  ```

  ```bash uv theme={null}
  uv add langchain langchain-text-splitters langchain-community bs4
  ```
</CodeGroup>

For more details, see our [Installation guide](/oss/python/langchain/install).

### LangSmith

Many of the applications you build with LangChain will contain multiple steps with multiple invocations of LLM calls. As these applications get more complex, it becomes crucial to be able to inspect what exactly is going on inside your chain or agent. The best way to do this is with [LangSmith](https://smith.langchain.com).

After you sign up at the link above, make sure to set your environment variables to start logging traces:

```shell  theme={null}
export LANGSMITH_TRACING="true"
export LANGSMITH_API_KEY="..."
```

Or, set them in Python:

```python  theme={null}
import getpass
import os

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = getpass.getpass()
```

### Components

We will need to select three components from LangChain's suite of integrations.

Select a chat model:

<Tabs>
  <Tab title="OpenAI">
    👉 Read the [OpenAI chat model integration docs](/oss/python/integrations/chat/openai/)

    ```shell  theme={null}
    pip install -U "langchain[openai]"
    ```

    <CodeGroup>
      ```python init_chat_model theme={null}
      import os
      from langchain.chat_models import init_chat_model

      os.environ["OPENAI_API_KEY"] = "sk-..."

      model = init_chat_model("gpt-4.1")
      ```

      ```python Model Class theme={null}
      import os
      from langchain_openai import ChatOpenAI

      os.environ["OPENAI_API_KEY"] = "sk-..."

      model = ChatOpenAI(model="gpt-4.1")
      ```
    </CodeGroup>
  </Tab>

  <Tab title="Anthropic">
    👉 Read the [Anthropic chat model integration docs](/oss/python/integrations/chat/anthropic/)

    ```shell  theme={null}
    pip install -U "langchain[anthropic]"
    ```

    <CodeGroup>
      ```python init_chat_model theme={null}
      import os
      from langchain.chat_models import init_chat_model

      os.environ["ANTHROPIC_API_KEY"] = "sk-..."

      model = init_chat_model("claude-sonnet-4-5-20250929")
      ```

      ```python Model Class theme={null}
      import os
      from langchain_anthropic import ChatAnthropic

      os.environ["ANTHROPIC_API_KEY"] = "sk-..."

      model = ChatAnthropic(model="claude-sonnet-4-5-20250929")
      ```
    </CodeGroup>
  </Tab>

  <Tab title="Azure">
    👉 Read the [Azure chat model integration docs](/oss/python/integrations/chat/azure_chat_openai/)

    ```shell  theme={null}
    pip install -U "langchain[openai]"
    ```

    <CodeGroup>
      ```python init_chat_model theme={null}
      import os
      from langchain.chat_models import init_chat_model

      os.environ["AZURE_OPENAI_API_KEY"] = "..."
      os.environ["AZURE_OPENAI_ENDPOINT"] = "..."
      os.environ["OPENAI_API_VERSION"] = "2025-03-01-preview"

      model = init_chat_model(
          "azure_openai:gpt-4.1",
          azure_deployment=os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"],
      )
      ```

      ```python Model Class theme={null}
      import os
      from langchain_openai import AzureChatOpenAI

      os.environ["AZURE_OPENAI_API_KEY"] = "..."
      os.environ["AZURE_OPENAI_ENDPOINT"] = "..."
      os.environ["OPENAI_API_VERSION"] = "2025-03-01-preview"

      model = AzureChatOpenAI(
          model="gpt-4.1",
          azure_deployment=os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"]
      )
      ```
    </CodeGroup>
  </Tab>

  <Tab title="Google Gemini">
    👉 Read the [Google GenAI chat model integration docs](/oss/python/integrations/chat/google_generative_ai/)

    ```shell  theme={null}
    pip install -U "langchain[google-genai]"
    ```

    <CodeGroup>
      ```python init_chat_model theme={null}
      import os
      from langchain.chat_models import init_chat_model

      os.environ["GOOGLE_API_KEY"] = "..."

      model = init_chat_model("google_genai:gemini-2.5-flash-lite")
      ```

      ```python Model Class theme={null}
      import os
      from langchain_google_genai import ChatGoogleGenerativeAI

      os.environ["GOOGLE_API_KEY"] = "..."

      model = ChatGoogleGenerativeAI(model="gemini-2.5-flash-lite")
      ```
    </CodeGroup>
  </Tab>

  <Tab title="AWS Bedrock">
    👉 Read the [AWS Bedrock chat model integration docs](/oss/python/integrations/chat/bedrock/)

    ```shell  theme={null}
    pip install -U "langchain[aws]"
    ```

    <CodeGroup>
      ```python init_chat_model theme={null}
      from langchain.chat_models import init_chat_model

      # Follow the steps here to configure your credentials:
      # https://docs.aws.amazon.com/bedrock/latest/userguide/getting-started.html

      model = init_chat_model(
          "anthropic.claude-3-5-sonnet-20240620-v1:0",
          model_provider="bedrock_converse",
      )
      ```

      ```python Model Class theme={null}
      from langchain_aws import ChatBedrock

      model = ChatBedrock(model="anthropic.claude-3-5-sonnet-20240620-v1:0")
      ```
    </CodeGroup>
  </Tab>

  <Tab title="HuggingFace">
    👉 Read the [HuggingFace chat model integration docs](/oss/python/integrations/chat/huggingface/)

    ```shell  theme={null}
    pip install -U "langchain[huggingface]"
    ```

    <CodeGroup>
      ```python init_chat_model theme={null}
      import os
      from langchain.chat_models import init_chat_model

      os.environ["HUGGINGFACEHUB_API_TOKEN"] = "hf_..."

      model = init_chat_model(
          "microsoft/Phi-3-mini-4k-instruct",
          model_provider="huggingface",
          temperature=0.7,
          max_tokens=1024,
      )
      ```

      ```python Model Class theme={null}
      import os
      from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint

      os.environ["HUGGINGFACEHUB_API_TOKEN"] = "hf_..."

      llm = HuggingFaceEndpoint(
          repo_id="microsoft/Phi-3-mini-4k-instruct",
          temperature=0.7,
          max_length=1024,
      )
      model = ChatHuggingFace(llm=llm)
      ```
    </CodeGroup>
  </Tab>
</Tabs>

Select an embeddings model:

<Tabs>
  <Tab title="OpenAI">
    ```shell  theme={null}
    pip install -U "langchain-openai"
    ```

    ```python  theme={null}
    import getpass
    import os

    if not os.environ.get("OPENAI_API_KEY"):
        os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

    from langchain_openai import OpenAIEmbeddings

    embeddings = OpenAIEmbeddings(model="text-embedding-3-large")
    ```
  </Tab>

  <Tab title="Azure">
    ```shell  theme={null}
    pip install -U "langchain-openai"
    ```

    ```python  theme={null}
    import getpass
    import os

    if not os.environ.get("AZURE_OPENAI_API_KEY"):
        os.environ["AZURE_OPENAI_API_KEY"] = getpass.getpass("Enter API key for Azure: ")

    from langchain_openai import AzureOpenAIEmbeddings

    embeddings = AzureOpenAIEmbeddings(
        azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
        azure_deployment=os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"],
        openai_api_version=os.environ["AZURE_OPENAI_API_VERSION"],
    )
    ```
  </Tab>

  <Tab title="Google Gemini">
    ```shell  theme={null}
    pip install -qU langchain-google-genai
    ```

    ```python  theme={null}
    import getpass
    import os

    if not os.environ.get("GOOGLE_API_KEY"):
        os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter API key for Google Gemini: ")

    from langchain_google_genai import GoogleGenerativeAIEmbeddings

    embeddings = GoogleGenerativeAIEmbeddings(model="models/gemini-embedding-001")
    ```
  </Tab>

  <Tab title="Google Vertex">
    ```shell  theme={null}
    pip install -qU langchain-google-vertexai
    ```

    ```python  theme={null}
    from langchain_google_vertexai import VertexAIEmbeddings

    embeddings = VertexAIEmbeddings(model="text-embedding-005")
    ```
  </Tab>

  <Tab title="AWS">
    ```shell  theme={null}
    pip install -qU langchain-aws
    ```

    ```python  theme={null}
    from langchain_aws import BedrockEmbeddings

    embeddings = BedrockEmbeddings(model_id="amazon.titan-embed-text-v2:0")
    ```
  </Tab>

  <Tab title="HuggingFace">
    ```shell  theme={null}
    pip install -qU langchain-huggingface
    ```

    ```python  theme={null}
    from langchain_huggingface import HuggingFaceEmbeddings

    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
    ```
  </Tab>

  <Tab title="Ollama">
    ```shell  theme={null}
    pip install -qU langchain-ollama
    ```

    ```python  theme={null}
    from langchain_ollama import OllamaEmbeddings

    embeddings = OllamaEmbeddings(model="llama3")
    ```
  </Tab>

  <Tab title="Cohere">
    ```shell  theme={null}
    pip install -qU langchain-cohere
    ```

    ```python  theme={null}
    import getpass
    import os

    if not os.environ.get("COHERE_API_KEY"):
        os.environ["COHERE_API_KEY"] = getpass.getpass("Enter API key for Cohere: ")

    from langchain_cohere import CohereEmbeddings

    embeddings = CohereEmbeddings(model="embed-english-v3.0")
    ```
  </Tab>

  <Tab title="MistralAI">
    ```shell  theme={null}
    pip install -qU langchain-mistralai
    ```

    ```python  theme={null}
    import getpass
    import os

    if not os.environ.get("MISTRALAI_API_KEY"):
        os.environ["MISTRALAI_API_KEY"] = getpass.getpass("Enter API key for MistralAI: ")

    from langchain_mistralai import MistralAIEmbeddings

    embeddings = MistralAIEmbeddings(model="mistral-embed")
    ```
  </Tab>

  <Tab title="Nomic">
    ```shell  theme={null}
    pip install -qU langchain-nomic
    ```

    ```python  theme={null}
    import getpass
    import os

    if not os.environ.get("NOMIC_API_KEY"):
        os.environ["NOMIC_API_KEY"] = getpass.getpass("Enter API key for Nomic: ")

    from langchain_nomic import NomicEmbeddings

    embeddings = NomicEmbeddings(model="nomic-embed-text-v1.5")
    ```
  </Tab>

  <Tab title="NVIDIA">
    ```shell  theme={null}
    pip install -qU langchain-nvidia-ai-endpoints
    ```

    ```python  theme={null}
    import getpass
    import os

    if not os.environ.get("NVIDIA_API_KEY"):
        os.environ["NVIDIA_API_KEY"] = getpass.getpass("Enter API key for NVIDIA: ")

    from langchain_nvidia_ai_endpoints import NVIDIAEmbeddings

    embeddings = NVIDIAEmbeddings(model="NV-Embed-QA")
    ```
  </Tab>

  <Tab title="Voyage AI">
    ```shell  theme={null}
    pip install -qU langchain-voyageai
    ```

    ```python  theme={null}
    import getpass
    import os

    if not os.environ.get("VOYAGE_API_KEY"):
        os.environ["VOYAGE_API_KEY"] = getpass.getpass("Enter API key for Voyage AI: ")

    from langchain-voyageai import VoyageAIEmbeddings

    embeddings = VoyageAIEmbeddings(model="voyage-3")
    ```
  </Tab>

  <Tab title="IBM watsonx">
    ```shell  theme={null}
    pip install -qU langchain-ibm
    ```

    ```python  theme={null}
    import getpass
    import os

    if not os.environ.get("WATSONX_APIKEY"):
        os.environ["WATSONX_APIKEY"] = getpass.getpass("Enter API key for IBM watsonx: ")

    from langchain_ibm import WatsonxEmbeddings

    embeddings = WatsonxEmbeddings(
        model_id="ibm/slate-125m-english-rtrvr",
        url="https://us-south.ml.cloud.ibm.com",
        project_id="<WATSONX PROJECT_ID>",
    )
    ```
  </Tab>

  <Tab title="Fake">
    ```shell  theme={null}
    pip install -qU langchain-core
    ```

    ```python  theme={null}
    from langchain_core.embeddings import DeterministicFakeEmbedding

    embeddings = DeterministicFakeEmbedding(size=4096)
    ```
  </Tab>

  <Tab title="Isaacus">
    ```shell  theme={null}
    pip install -qU langchain-isaacus
    ```

    ```python  theme={null}
    import getpass
    import os

    if not os.environ.get("ISAACUS_API_KEY"):
    os.environ["ISAACUS_API_KEY"] = getpass.getpass("Enter API key for Isaacus: ")

    from langchain_isaacus import IsaacusEmbeddings

    embeddings = IsaacusEmbeddings(model="kanon-2-embedder")
    ```
  </Tab>
</Tabs>

Select a vector store:

<Tabs>
  <Tab title="In-memory">
    ```shell  theme={null}
    pip install -U "langchain-core"
    ```

    ```python  theme={null}
    from langchain_core.vectorstores import InMemoryVectorStore

    vector_store = InMemoryVectorStore(embeddings)
    ```
  </Tab>

  <Tab title="Amazon OpenSearch">
    ```shell  theme={null}
    pip install -qU  boto3
    ```

    ```python  theme={null}
    from opensearchpy import RequestsHttpConnection

    service = "es"  # must set the service as 'es'
    region = "us-east-2"
    credentials = boto3.Session(
        aws_access_key_id="xxxxxx", aws_secret_access_key="xxxxx"
    ).get_credentials()
    awsauth = AWS4Auth("xxxxx", "xxxxxx", region, service, session_token=credentials.token)

    vector_store = OpenSearchVectorSearch.from_documents(
        docs,
        embeddings,
        opensearch_url="host url",
        http_auth=awsauth,
        timeout=300,
        use_ssl=True,
        verify_certs=True,
        connection_class=RequestsHttpConnection,
        index_name="test-index",
    )
    ```
  </Tab>

  <Tab title="AstraDB">
    ```shell  theme={null}
    pip install -U "langchain-astradb"
    ```

    ```python  theme={null}
    from langchain_astradb import AstraDBVectorStore

    vector_store = AstraDBVectorStore(
        embedding=embeddings,
        api_endpoint=ASTRA_DB_API_ENDPOINT,
        collection_name="astra_vector_langchain",
        token=ASTRA_DB_APPLICATION_TOKEN,
        namespace=ASTRA_DB_NAMESPACE,
    )
    ```
  </Tab>

  <Tab title="Chroma">
    ```shell  theme={null}
    pip install -qU langchain-chroma
    ```

    ```python  theme={null}
    from langchain_chroma import Chroma

    vector_store = Chroma(
        collection_name="example_collection",
        embedding_function=embeddings,
        persist_directory="./chroma_langchain_db",  # Where to save data locally, remove if not necessary
    )
    ```
  </Tab>

  <Tab title="FAISS">
    ```shell  theme={null}
    pip install -qU langchain-community faiss-cpu
    ```

    ```python  theme={null}
    import faiss
    from langchain_community.docstore.in_memory import InMemoryDocstore
    from langchain_community.vectorstores import FAISS

    embedding_dim = len(embeddings.embed_query("hello world"))
    index = faiss.IndexFlatL2(embedding_dim)

    vector_store = FAISS(
        embedding_function=embeddings,
        index=index,
        docstore=InMemoryDocstore(),
        index_to_docstore_id={},
    )
    ```
  </Tab>

  <Tab title="Milvus">
    ```shell  theme={null}
    pip install -qU langchain-milvus
    ```

    ```python  theme={null}
    from langchain_milvus import Milvus

    URI = "./milvus_example.db"

    vector_store = Milvus(
        embedding_function=embeddings,
        connection_args={"uri": URI},
        index_params={"index_type": "FLAT", "metric_type": "L2"},
    )
    ```
  </Tab>

  <Tab title="MongoDB">
    ```shell  theme={null}
    pip install -qU langchain-mongodb
    ```

    ```python  theme={null}
    from langchain_mongodb import MongoDBAtlasVectorSearch

    vector_store = MongoDBAtlasVectorSearch(
        embedding=embeddings,
        collection=MONGODB_COLLECTION,
        index_name=ATLAS_VECTOR_SEARCH_INDEX_NAME,
        relevance_score_fn="cosine",
    )
    ```
  </Tab>

  <Tab title="PGVector">
    ```shell  theme={null}
    pip install -qU langchain-postgres
    ```

    ```python  theme={null}
    from langchain_postgres import PGVector

    vector_store = PGVector(
        embeddings=embeddings,
        collection_name="my_docs",
        connection="postgresql+psycopg://...",
    )
    ```
  </Tab>

  <Tab title="PGVectorStore">
    ```shell  theme={null}
    pip install -qU langchain-postgres
    ```

    ```python  theme={null}
    from langchain_postgres import PGEngine, PGVectorStore

    pg_engine = PGEngine.from_connection_string(
        url="postgresql+psycopg://..."
    )

    vector_store = PGVectorStore.create_sync(
        engine=pg_engine,
        table_name='test_table',
        embedding_service=embeddings
    )
    ```
  </Tab>

  <Tab title="Pinecone">
    ```shell  theme={null}
    pip install -qU langchain-pinecone
    ```

    ```python  theme={null}
    from langchain_pinecone import PineconeVectorStore
    from pinecone import Pinecone

    pc = Pinecone(api_key=...)
    index = pc.Index(index_name)

    vector_store = PineconeVectorStore(embedding=embeddings, index=index)
    ```
  </Tab>

  <Tab title="Qdrant">
    ```shell  theme={null}
    pip install -qU langchain-qdrant
    ```

    ```python  theme={null}
    from qdrant_client.models import Distance, VectorParams
    from langchain_qdrant import QdrantVectorStore
    from qdrant_client import QdrantClient

    client = QdrantClient(":memory:")

    vector_size = len(embeddings.embed_query("sample text"))

    if not client.collection_exists("test"):
        client.create_collection(
            collection_name="test",
            vectors_config=VectorParams(size=vector_size, distance=Distance.COSINE)
        )
    vector_store = QdrantVectorStore(
        client=client,
        collection_name="test",
        embedding=embeddings,
    )
    ```
  </Tab>
</Tabs>

## 1. Indexing

<Note>
  **This section is an abbreviated version of the content in the [semantic search tutorial](/oss/python/langchain/knowledge-base).**

  If your data is already indexed and available for search (i.e., you have a function to execute a search), or if you're comfortable with [document loaders](/oss/python/langchain/retrieval#document_loaders), [embeddings](/oss/python/langchain/retrieval#embedding_models), and [vector stores](/oss/python/langchain/retrieval#vectorstores), feel free to skip to the next section on [retrieval and generation](/oss/python/langchain/rag#2-retrieval-and-generation).
</Note>

Indexing commonly works as follows:

1. **Load**: First we need to load our data. This is done with [Document Loaders](/oss/python/langchain/retrieval#document_loaders).
2. **Split**: [Text splitters](/oss/python/langchain/retrieval#text_splitters) break large `Documents` into smaller chunks. This is useful both for indexing data and passing it into a model, as large chunks are harder to search over and won't fit in a model's finite context window.
3. **Store**: We need somewhere to store and index our splits, so that they can be searched over later. This is often done using a [VectorStore](/oss/python/langchain/retrieval#vectorstores) and [Embeddings](/oss/python/langchain/retrieval#embedding_models) model.

<img src="https://mintcdn.com/langchain-5e9cc07a/I6RpA28iE233vhYX/images/rag_indexing.png?fit=max&auto=format&n=I6RpA28iE233vhYX&q=85&s=21403ce0d0c772da84dcc5b75cff4451" alt="index_diagram" data-og-width="2583" width="2583" data-og-height="1299" height="1299" data-path="images/rag_indexing.png" data-optimize="true" data-opv="3" srcset="https://mintcdn.com/langchain-5e9cc07a/I6RpA28iE233vhYX/images/rag_indexing.png?w=280&fit=max&auto=format&n=I6RpA28iE233vhYX&q=85&s=bf4eb8255b82a809dbbd2bc2a96d2ed7 280w, https://mintcdn.com/langchain-5e9cc07a/I6RpA28iE233vhYX/images/rag_indexing.png?w=560&fit=max&auto=format&n=I6RpA28iE233vhYX&q=85&s=4ebc538b2c4765b609f416025e4dbbda 560w, https://mintcdn.com/langchain-5e9cc07a/I6RpA28iE233vhYX/images/rag_indexing.png?w=840&fit=max&auto=format&n=I6RpA28iE233vhYX&q=85&s=1838328a870c7353c42bf1cc2290a779 840w, https://mintcdn.com/langchain-5e9cc07a/I6RpA28iE233vhYX/images/rag_indexing.png?w=1100&fit=max&auto=format&n=I6RpA28iE233vhYX&q=85&s=675f55e100bab5e2904d27db01775ccc 1100w, https://mintcdn.com/langchain-5e9cc07a/I6RpA28iE233vhYX/images/rag_indexing.png?w=1650&fit=max&auto=format&n=I6RpA28iE233vhYX&q=85&s=4b9e544a7a3ec168651558bce854eb60 1650w, https://mintcdn.com/langchain-5e9cc07a/I6RpA28iE233vhYX/images/rag_indexing.png?w=2500&fit=max&auto=format&n=I6RpA28iE233vhYX&q=85&s=f5aeaaaea103128f374c03b05a317263 2500w" />

### Loading documents

We need to first load the blog post contents. We can use [DocumentLoaders](/oss/python/langchain/retrieval#document_loaders) for this, which are objects that load in data from a source and return a list of [Document](https://reference.langchain.com/python/langchain_core/documents/#langchain_core.documents.base.Document) objects.

In this case we'll use the [`WebBaseLoader`](/oss/python/integrations/document_loaders/web_base), which uses `urllib` to load HTML from web URLs and `BeautifulSoup` to parse it to text. We can customize the HTML -> text parsing by passing in parameters into the `BeautifulSoup` parser via `bs_kwargs` (see [BeautifulSoup docs](https://beautiful-soup-4.readthedocs.io/en/latest/#beautifulsoup)). In this case only HTML tags with class “post-content”, “post-title”, or “post-header” are relevant, so we'll remove all others.

```python  theme={null}
import bs4
from langchain_community.document_loaders import WebBaseLoader

# Only keep post title, headers, and content from the full HTML.
bs4_strainer = bs4.SoupStrainer(class_=("post-title", "post-header", "post-content"))
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs={"parse_only": bs4_strainer},
)
docs = loader.load()

assert len(docs) == 1
print(f"Total characters: {len(docs[0].page_content)}")
```

```text  theme={null}
Total characters: 43131
```

```python  theme={null}
print(docs[0].page_content[:500])
```

```text  theme={null}
      LLM Powered Autonomous Agents

Date: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng


Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.
Agent System Overview#
In
```

**Go deeper**

`DocumentLoader`: Object that loads data from a source as list of `Documents`.

* [Integrations](/oss/python/integrations/document_loaders/): 160+ integrations to choose from.
* [`BaseLoader`](https://reference.langchain.com/python/langchain_core/document_loaders/#langchain_core.document_loaders.BaseLoader): API reference for the base interface.

### Splitting documents

Our loaded document is over 42k characters which is too long to fit into the context window of many models. Even for those models that could fit the full post in their context window, models can struggle to find information in very long inputs.

To handle this we'll split the [`Document`](https://reference.langchain.com/python/langchain_core/documents/#langchain_core.documents.base.Document) into chunks for embedding and vector storage. This should help us retrieve only the most relevant parts of the blog post at run time.

As in the [semantic search tutorial](/oss/python/langchain/knowledge-base), we use a `RecursiveCharacterTextSplitter`, which will recursively split the document using common separators like new lines until each chunk is the appropriate size. This is the recommended text splitter for generic text use cases.

```python  theme={null}
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # chunk size (characters)
    chunk_overlap=200,  # chunk overlap (characters)
    add_start_index=True,  # track index in original document
)
all_splits = text_splitter.split_documents(docs)

print(f"Split blog post into {len(all_splits)} sub-documents.")
```

```text  theme={null}
Split blog post into 66 sub-documents.
```

**Go deeper**

`TextSplitter`: Object that splits a list of [`Document`](https://reference.langchain.com/python/langchain_core/documents/#langchain_core.documents.base.Document) objects into smaller
chunks for storage and retrieval.

* [Integrations](/oss/python/integrations/splitters/)
* [Interface](https://python.langchain.com/api_reference/text_splitters/base/langchain_text_splitters.base.TextSplitter.html): API reference for the base interface.

### Storing documents

Now we need to index our 66 text chunks so that we can search over them at runtime. Following the [semantic search tutorial](/oss/python/langchain/knowledge-base), our approach is to [embed](/oss/python/langchain/retrieval#embedding_models/) the contents of each document split and insert these embeddings into a [vector store](/oss/python/langchain/retrieval#vectorstores/). Given an input query, we can then use vector search to retrieve relevant documents.

We can embed and store all of our document splits in a single command using the vector store and embeddings model selected at the [start of the tutorial](/oss/python/langchain/rag#components).

```python  theme={null}
document_ids = vector_store.add_documents(documents=all_splits)

print(document_ids[:3])
```

```python  theme={null}
['07c18af6-ad58-479a-bfb1-d508033f9c64', '9000bf8e-1993-446f-8d4d-f4e507ba4b8f', 'ba3b5d14-bed9-4f5f-88be-44c88aedc2e6']
```

**Go deeper**

`Embeddings`: Wrapper around a text embedding model, used for converting text to embeddings.

* [Integrations](/oss/python/integrations/text_embedding/): 30+ integrations to choose from.
* [Interface](https://reference.langchain.com/python/langchain_core/embeddings/#langchain_core.embeddings.embeddings.Embeddings): API reference for the base interface.

`VectorStore`: Wrapper around a vector database, used for storing and querying embeddings.

* [Integrations](/oss/python/integrations/vectorstores/): 40+ integrations to choose from.
* [Interface](https://python.langchain.com/api_reference/core/vectorstores/langchain_core.vectorstores.base.VectorStore.html): API reference for the base interface.

This completes the **Indexing** portion of the pipeline. At this point we have a query-able vector store containing the chunked contents of our blog post. Given a user question, we should ideally be able to return the snippets of the blog post that answer the question.

## 2. Retrieval and Generation

RAG applications commonly work as follows:

1. **Retrieve**: Given a user input, relevant splits are retrieved from storage using a [Retriever](/oss/python/langchain/retrieval#retrievers).
2. **Generate**: A [model](/oss/python/langchain/models) produces an answer using a prompt that includes both the question with the retrieved data

<img src="https://mintcdn.com/langchain-5e9cc07a/I6RpA28iE233vhYX/images/rag_retrieval_generation.png?fit=max&auto=format&n=I6RpA28iE233vhYX&q=85&s=994c3585cece93c80873d369960afd44" alt="retrieval_diagram" data-og-width="2532" width="2532" data-og-height="1299" height="1299" data-path="images/rag_retrieval_generation.png" data-optimize="true" data-opv="3" srcset="https://mintcdn.com/langchain-5e9cc07a/I6RpA28iE233vhYX/images/rag_retrieval_generation.png?w=280&fit=max&auto=format&n=I6RpA28iE233vhYX&q=85&s=3bd28b3662e08c8364b60b74f510751e 280w, https://mintcdn.com/langchain-5e9cc07a/I6RpA28iE233vhYX/images/rag_retrieval_generation.png?w=560&fit=max&auto=format&n=I6RpA28iE233vhYX&q=85&s=43484903ca631a47a54e86191eb5ba22 560w, https://mintcdn.com/langchain-5e9cc07a/I6RpA28iE233vhYX/images/rag_retrieval_generation.png?w=840&fit=max&auto=format&n=I6RpA28iE233vhYX&q=85&s=67fe2302e241fc24238a5df1cf56573d 840w, https://mintcdn.com/langchain-5e9cc07a/I6RpA28iE233vhYX/images/rag_retrieval_generation.png?w=1100&fit=max&auto=format&n=I6RpA28iE233vhYX&q=85&s=d390a6a758e688ec36352d30b22249b0 1100w, https://mintcdn.com/langchain-5e9cc07a/I6RpA28iE233vhYX/images/rag_retrieval_generation.png?w=1650&fit=max&auto=format&n=I6RpA28iE233vhYX&q=85&s=59729377317a0631598b6a4a2a7d8c92 1650w, https://mintcdn.com/langchain-5e9cc07a/I6RpA28iE233vhYX/images/rag_retrieval_generation.png?w=2500&fit=max&auto=format&n=I6RpA28iE233vhYX&q=85&s=c07711c71153c3b2dfd5b0104ad3e324 2500w" />

Now let's write the actual application logic. We want to create a simple application that takes a user question, searches for documents relevant to that question, passes the retrieved documents and initial question to a model, and returns an answer.

We will demonstrate:

1. A RAG [agent](#rag-agents) that executes searches with a simple tool. This is a good general-purpose implementation.
2. A two-step RAG [chain](#rag-chains) that uses just a single LLM call per query. This is a fast and effective method for simple queries.

### RAG agents

One formulation of a RAG application is as a simple [agent](/oss/python/langchain/agents) with a tool that retrieves information. We can assemble a minimal RAG agent by implementing a [tool](/oss/python/langchain/tools) that wraps our vector store:

```python  theme={null}
from langchain.tools import tool

@tool(response_format="content_and_artifact")
def retrieve_context(query: str):
    """Retrieve information to help answer a query."""
    retrieved_docs = vector_store.similarity_search(query, k=2)
    serialized = "\n\n".join(
        (f"Source: {doc.metadata}\nContent: {doc.page_content}")
        for doc in retrieved_docs
    )
    return serialized, retrieved_docs
```

<Tip>
  Here we use the [tool decorator](https://reference.langchain.com/python/langchain/tools/#langchain.tools.tool) to configure the tool to attach raw documents as [artifacts](/oss/python/langchain/messages#param-artifact) to each [ToolMessage](/oss/python/langchain/messages#tool-message). This will let us access document metadata in our application, separate from the stringified representation that is sent to the model.
</Tip>

<Tip>
  Retrieval tools are not limited to a single string `query` argument, as in the above example. You can
  force the LLM to specify additional search parameters by adding arguments— for example, a category:

  ```python  theme={null}
  from typing import Literal

  def retrieve_context(query: str, section: Literal["beginning", "middle", "end"]):
  ```
</Tip>

Given our tool, we can construct the agent:

```python  theme={null}
from langchain.agents import create_agent


tools = [retrieve_context]
# If desired, specify custom instructions
prompt = (
    "You have access to a tool that retrieves context from a blog post. "
    "Use the tool to help answer user queries."
)
agent = create_agent(model, tools, system_prompt=prompt)
```

Let's test this out. We construct a question that would typically require an iterative sequence of retrieval steps to answer:

```python  theme={null}
query = (
    "What is the standard method for Task Decomposition?\n\n"
    "Once you get the answer, look up common extensions of that method."
)

for event in agent.stream(
    {"messages": [{"role": "user", "content": query}]},
    stream_mode="values",
):
    event["messages"][-1].pretty_print()
```

```
================================ Human Message =================================

What is the standard method for Task Decomposition?

Once you get the answer, look up common extensions of that method.
================================== Ai Message ==================================
Tool Calls:
  retrieve_context (call_d6AVxICMPQYwAKj9lgH4E337)
 Call ID: call_d6AVxICMPQYwAKj9lgH4E337
  Args:
    query: standard method for Task Decomposition
================================= Tool Message =================================
Name: retrieve_context

Source: {'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}
Content: Task decomposition can be done...

Source: {'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}
Content: Component One: Planning...
================================== Ai Message ==================================
Tool Calls:
  retrieve_context (call_0dbMOw7266jvETbXWn4JqWpR)
 Call ID: call_0dbMOw7266jvETbXWn4JqWpR
  Args:
    query: common extensions of the standard method for Task Decomposition
================================= Tool Message =================================
Name: retrieve_context

Source: {'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}
Content: Task decomposition can be done...

Source: {'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}
Content: Component One: Planning...
================================== Ai Message ==================================

The standard method for Task Decomposition often used is the Chain of Thought (CoT)...
```

Note that the agent:

1. Generates a query to search for a standard method for task decomposition;
2. Receiving the answer, generates a second query to search for common extensions of it;
3. Having received all necessary context, answers the question.

We can see the full sequence of steps, along with latency and other metadata, in the [LangSmith trace](https://smith.langchain.com/public/7b42d478-33d2-4631-90a4-7cb731681e88/r).

<Tip>
  You can add a deeper level of control and customization using the [LangGraph](/oss/python/langgraph/overview) framework directly— for example, you can add steps to grade document relevance and rewrite search queries. Check out LangGraph's [Agentic RAG tutorial](/oss/python/langgraph/agentic-rag) for more advanced formulations.
</Tip>

### RAG chains

In the above [agentic RAG](#rag-agents) formulation we allow the LLM to use its discretion in generating a [tool call](/oss/python/langchain/models#tool-calling) to help answer user queries. This is a good general-purpose solution, but comes with some trade-offs:

| ✅ Benefits                                                                                                                                                 | ⚠️ Drawbacks                                                                                                                                |
| ---------------------------------------------------------------------------------------------------------------------------------------------------------- | ------------------------------------------------------------------------------------------------------------------------------------------- |
| **Search only when needed** – The LLM can handle greetings, follow-ups, and simple queries without triggering unnecessary searches.                        | **Two inference calls** – When a search is performed, it requires one call to generate the query and another to produce the final response. |
| **Contextual search queries** – By treating search as a tool with a `query` input, the LLM crafts its own queries that incorporate conversational context. | **Reduced control** – The LLM may skip searches when they are actually needed, or issue extra searches when unnecessary.                    |
| **Multiple searches allowed** – The LLM can execute several searches in support of a single user query.                                                    |                                                                                                                                             |

Another common approach is a two-step chain, in which we always run a search (potentially using the raw user query) and incorporate the result as context for a single LLM query. This results in a single inference call per query, buying reduced latency at the expense of flexibility.

In this approach we no longer call the model in a loop, but instead make a single pass.

We can implement this chain by removing tools from the agent and instead incorporating the retrieval step into a custom prompt:

```python  theme={null}
from langchain.agents.middleware import dynamic_prompt, ModelRequest

@dynamic_prompt
def prompt_with_context(request: ModelRequest) -> str:
    """Inject context into state messages."""
    last_query = request.state["messages"][-1].text
    retrieved_docs = vector_store.similarity_search(last_query)

    docs_content = "\n\n".join(doc.page_content for doc in retrieved_docs)

    system_message = (
        "You are a helpful assistant. Use the following context in your response:"
        f"\n\n{docs_content}"
    )

    return system_message


agent = create_agent(model, tools=[], middleware=[prompt_with_context])
```

Let's try this out:

```python  theme={null}
query = "What is task decomposition?"
for step in agent.stream(
    {"messages": [{"role": "user", "content": query}]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()
```

```
================================ Human Message =================================

What is task decomposition?
================================== Ai Message ==================================

Task decomposition is...
```

In the [LangSmith trace](https://smith.langchain.com/public/0322904b-bc4c-4433-a568-54c6b31bbef4/r/9ef1c23e-380e-46bf-94b3-d8bb33df440c) we can see the retrieved context incorporated into the model prompt.

This is a fast and effective method for simple queries in constrained settings, when we typically do want to run user queries through semantic search to pull additional context.

<Accordion title="Returning source documents">
  The above [RAG chain](#rag-chains) incorporates retrieved context into a single system message for that run.

  As in the [agentic RAG](#rag-agents) formulation, we sometimes want to include raw source documents in the application state to have access to document metadata. We can do this for the two-step chain case by:

  1. Adding a key to the state to store the retrieved documents
  2. Adding a new node via a [pre-model hook](/oss/python/langchain/agents#pre-model-hook) to populate that key (as well as inject the context).

  ```python  theme={null}
  from typing import Any
  from langchain_core.documents import Document
  from langchain.agents.middleware import AgentMiddleware, AgentState


  class State(AgentState):
      context: list[Document]


  class RetrieveDocumentsMiddleware(AgentMiddleware[State]):
      state_schema = State

      def before_model(self, state: AgentState) -> dict[str, Any] | None:
          last_message = state["messages"][-1]
          retrieved_docs = vector_store.similarity_search(last_message.text)

          docs_content = "\n\n".join(doc.page_content for doc in retrieved_docs)

          augmented_message_content = (
              f"{last_message.text}\n\n"
              "Use the following context to answer the query:\n"
              f"{docs_content}"
          )
          return {
              "messages": [last_message.model_copy(update={"content": augmented_message_content})],
              "context": retrieved_docs,
          }


  agent = create_agent(
      model,
      tools=[],
      middleware=[RetrieveDocumentsMiddleware()],
  )
  ```
</Accordion>

## Next steps

Now that we've implemented a simple RAG application via [`create_agent`](https://reference.langchain.com/python/langchain/agents/#langchain.agents.create_agent), we can easily incorporate new features and go deeper:

* [Stream](/oss/python/langchain/streaming) tokens and other information for responsive user experiences
* Add [conversational memory](/oss/python/langchain/short-term-memory) to support multi-turn interactions
* Add [long-term memory](/oss/python/langchain/long-term-memory) to support memory across conversational threads
* Add [structured responses](/oss/python/langchain/structured-output)
* Deploy your application with [LangSmith Deployment](/langsmith/deployments)

***

<Callout icon="pen-to-square" iconType="regular">
  [Edit this page on GitHub](https://github.com/langchain-ai/docs/edit/main/src/oss/langchain/rag.mdx) or [file an issue](https://github.com/langchain-ai/docs/issues/new/choose).
</Callout>

<Tip icon="terminal" iconType="regular">
  [Connect these docs](/use-these-docs) to Claude, VSCode, and more via MCP for real-time answers.
</Tip>


---

> To find navigation and other pages in this documentation, fetch the llms.txt file at: https://docs.langchain.com/llms.txt'''

Split the document text into manageble chunks

In [4]:
text_splitter = CharacterTextSplitter(chunk_size=300, chunk_overlap=10)
texts = text_splitter.split_text(documents)

Created a chunk of size 310, which is longer than the specified 300
Created a chunk of size 301, which is longer than the specified 300
Created a chunk of size 337, which is longer than the specified 300
Created a chunk of size 316, which is longer than the specified 300
Created a chunk of size 371, which is longer than the specified 300
Created a chunk of size 325, which is longer than the specified 300
Created a chunk of size 328, which is longer than the specified 300
Created a chunk of size 362, which is longer than the specified 300
Created a chunk of size 469, which is longer than the specified 300
Created a chunk of size 680, which is longer than the specified 300
Created a chunk of size 1372, which is longer than the specified 300
Created a chunk of size 330, which is longer than the specified 300
Created a chunk of size 522, which is longer than the specified 300
Created a chunk of size 314, which is longer than the specified 300
Created a chunk of size 381, which is longer th

Setup Hugging Face Embeddings

In [5]:
# Define the model name to be used for generating embeddings
model_name = "sentence-transformers/all-mpnet-base-v2"

# Define any additional keyword arguments for the model and encoding process
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}

# Create the HuggingFaceEmbeddings instance with the specified model and parameters
# These embeddings will be used to convert text into vector representations
hf_embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

Create FAISS index for efficient similartry search

In [6]:
sample_embedding = np.array(hf_embeddings.embed_query("Sample text"))
# print(sample_embedding)
dimension = sample_embedding.shape[0]
index = faiss.IndexFlatL2(dimension)

Create FAISS vector store with Hugging Face embedding function

In [7]:
vector_store = FAISS(
    embedding_function=hf_embeddings.embed_query,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)

vector_store.add_texts(texts)

`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.


['b388536c-eb7f-4cfa-9581-297bfd60aae3',
 'a7478934-1aa8-43eb-834f-1b82f12a5f22',
 'ccbf6bc6-cc32-4d6f-9761-ae55277eaf13',
 '4b86a732-2ced-4e68-bdc8-5d2a6a1655eb',
 '3cb0f067-c9a4-469f-994e-f0981c9db77d',
 'a706fc7b-cc77-4b89-a1ea-adfddb8bbadb',
 '002234ef-3308-49bb-ba76-279956aedc15',
 'aefc9661-b123-4a16-a42c-1f18202b569e',
 'e7b2df10-6dac-4a1d-976f-eeba90e04b53',
 '5023aec4-208e-4cdc-9c76-a48918dece9e',
 'c44c62b4-62fe-40c9-84a9-ed50876195dc',
 'ca366822-f26a-441b-b5ff-efa620c5f69a',
 'ac77bf6a-681d-4e81-9654-9e251557d424',
 'fd13b7c3-dc61-4822-8d15-422de56d522e',
 '4e174107-43e0-44ad-b317-b68ad07bd3a7',
 'eb6594b1-3492-4a1c-96d0-3380728f84b0',
 '959a88bc-4935-405e-ac9b-01a5e6ebda2b',
 '33ac2601-6fd4-4964-b145-321e5458abe7',
 '335fef85-c0bc-48b9-b825-e6787e708e3f',
 '3397fda7-bb0b-4b98-b1d2-e9be51b5e51a',
 '80843e25-7c4b-4283-af5d-c2282c6d0ae7',
 '7f6eaca8-6206-4417-b4dc-4d3fc6824727',
 'c44c6bbb-90a9-4f67-b67c-b84182b79b93',
 'b002bd7a-c1e1-4332-871d-63dfa72e0dbb',
 '733a4419-686f-

Setup the Language Model (LLM) for Question Answering (QA)

In [24]:
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint

llm_endpoint = HuggingFaceEndpoint(
    repo_id="meta-llama/Meta-Llama-3-8B-Instruct",
    provider="novita",
    huggingfacehub_api_token=HUGGINGFACE_API_TOKEN,
    temperature=0.6,
)

llm = ChatHuggingFace(llm=llm_endpoint)

Create a Retrieval-based Question Answering (QA) system

In [25]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template(
    """Answer the question using the context below.
If you don't know the answer, say you don't know.

Context:
{context}

Question:
{question}"""
)

In [ ]:
retriever=vector_store.as_retriever()
from langchain_core.runnables import RunnablePassthrough

qa = (
    {
        "context": retriever,
        "question": RunnablePassthrough(),
    }
    | prompt
    | llm
)

Generation Phase

In [31]:
answer = qa.invoke("How to create a simple indexing pipeline")
print(answer.content)

According to the provided context, a simple indexing pipeline can be created in ~40 lines of code. However, the full code snippet is not provided in the given context. Therefore, I don't know the exact steps to create a simple indexing pipeline.
